In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.metrics import r2_score

# Load the new training data
new_data = pd.read_excel("153Satir.xlsx")
new_data = new_data.drop(["id", "temp", "w_status"], axis=1)
new_data['date_v'] = pd.to_datetime(new_data['date_v'])
new_data['unix_time'] = new_data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))
new_data = new_data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()
new_data['yukh_scaled'] = scaler_yukh.fit_transform(new_data[['yukh']])
new_data['reg_scaled'] = scaler_reg.fit_transform(new_data[['reg']])
new_data['power_scaled'] = scaler_power.fit_transform(new_data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
new_data = new_data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = new_data['yukh_scaled'][i:i+window_size].values
        window_reg = new_data['reg_scaled'][i:i+window_size].values
        window_power = new_data['power_scaled'][i:i+window_size].values
        target = new_data['yukh_scaled'][i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme
window_size = 64
# Create time series data for the new training set
X_new, y_new = create_time_series_data(new_data, window_size)

# Split the new data into training and testing sets
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.25, random_state=42)

# Load the pre-trained model
model = load_model('retrained_model.h5')

# Retrain the model with the new data
model.fit(X_train_new, y_train_new, epochs=100, batch_size=16, validation_data=(X_test_new, y_test_new))

# Save the retrained model
model.save('retrained_model.h5')

# Test the retrained model on a separate test set if needed
test_predictions_new = model.predict(X_test_new)
r2_test_new = r2_score(y_test_new, test_predictions_new)

print(f'Retrained Model Test R-squared score: {r2_test_new}')


Epoch 1/100
5/5 [==============================] - 4s 227ms/step - loss: 0.0450 - val_loss: 0.0033
Epoch 2/100
5/5 [==============================] - 0s 91ms/step - loss: 0.0142 - val_loss: 0.0031
Epoch 3/100
5/5 [==============================] - 0s 92ms/step - loss: 0.0151 - val_loss: 0.0083
Epoch 4/100
5/5 [==============================] - 0s 99ms/step - loss: 0.0087 - val_loss: 0.0056
Epoch 5/100
5/5 [==============================] - 0s 89ms/step - loss: 0.0040 - val_loss: 0.0059
Epoch 6/100
5/5 [==============================] - 0s 98ms/step - loss: 0.0053 - val_loss: 0.0055
Epoch 7/100
5/5 [==============================] - 0s 85ms/step - loss: 0.0050 - val_loss: 0.0049
Epoch 8/100
5/5 [==============================] - 0s 87ms/step - loss: 0.0041 - val_loss: 0.0048
Epoch 9/100
5/5 [==============================] - 0s 88ms/step - loss: 0.0051 - val_loss: 0.0041
Epoch 10/100
5/5 [==============================] - 0s 88ms/step - loss: 0.0046 - val_loss: 0.0037
Epoch 11/100
5/5 [

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 692ms/step
Retrained Model Test R-squared score: 0.7426882195545983


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import load_model

# Veriyi yükleme ve gereksiz sütunları kaldırma
data = pd.read_excel("153Satir.xlsx")
data = data.drop(["id", "temp", "w_status"], axis=1)

data = data[(data != 0).all(axis=1)]  # Tüm sütunlar için 0 olmayan satırları seçme
data = data.dropna()  # NaN değerleri içeren satırları kaldırma

# 'date' sütununu datetime formatına çevirme
data['date_v'] = pd.to_datetime(data['date_v'])

# 'date' sütununu Unix zaman damgasına çevirme
data['unix_time'] = data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))

# Gereksiz sütunu ve orijinal tarih sütununu kaldırma
data = data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()

data['yukh_scaled'] = scaler_yukh.fit_transform(data[['yukh']])
data['reg_scaled'] = scaler_reg.fit_transform(data[['reg']])
data['power_scaled'] = scaler_power.fit_transform(data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
data = data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = data['yukh_scaled'][i:i+window_size].values
        window_reg = data['reg_scaled'][i:i+window_size].values
        window_power = data['power_scaled'][i:i+window_size].values
        target = data['yukh_scaled'][i: i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme (aynı boyutu kullanmalısınız)
window_size = 64

# Giriş ve hedef verilerini oluşturma
X, y = create_time_series_data(data, window_size)

# Modeli yükleme
model = load_model('retrained_model.h5')

# Test seti üzerinde tahmin yapma
test_predictions = model.predict(X)

# Test setindeki pencere boyutunda son gerçek değerler
y_true = y[:, -1]

# Test setindeki pencere boyutunda son tahminler
y_pred = test_predictions[:, -1]

# R-kare skoru hesaplama
r2_test = r2_score(y_true, y_pred)

print(f'Modelin Test R-kare skoru: {r2_test}')

3/3 [==============================] - 0s 23ms/step
Modelin Test R-kare skoru: 0.8118045305720263


In [ ]:
# Gerçek 'yukh' değerlerini al
yukh_original = scaler_yukh.inverse_transform(y[:, -1].reshape(-1, 1)).flatten()

# Tahmin edilen 'yukh' değerlerini ölçeklendirmeyi tersine çevirme
predicted_yukh_original = scaler_yukh.inverse_transform(test_predictions[:, -1].reshape(-1, 1)).flatten()

# Uzunlukları kontrol et
print(len(yukh_original), len(predicted_yukh_original))

# Veri çerçevesi oluşturma
result_df = pd.DataFrame({'Gerçek Yukh': yukh_original, 'Tahmin Yukh': predicted_yukh_original})

# Sonuçları ekrana yazdırma
print(result_df)

# Veriyi Excel dosyasına yazma
result_df.to_excel("gercek_tahmin_yukh1.xlsx", index=False)


90 90
    Gerçek Yukh  Tahmin Yukh
0        5989.0  5971.255859
1        5990.0  5971.262695
2        5991.0  5971.427734
3        5989.0  5971.347656
4        5990.0  5971.528809
..          ...          ...
85       6026.0  5999.142090
86       6027.0  5999.371582
87       6028.0  5999.753906
88       6029.0  6000.270508
89       6029.0  6000.573242

[90 rows x 2 columns]
